# CHAPTER2 データの登録と更新

## 2.7 基本データのバインディング

DOM更新などをバインディングを行う際には使われるデータは`リアクティブデータ`として定義される必要がある。
Vue.jsではこの`リアクティブデータ`が更新された時のみ、DOMの自動更新が行われる。
* リアクティブデータとは
    * Vue.jsが監視しているデータのこと
        * ここで言う監視とは、代入や取得が行われたときフック処理が登録されていることを指す。
    * つまり、フック処理が登録されている`反応できるデータ`のことを`リアクティブデータ`と呼ぶ。

この章では、ディレクティブを用いたデータバインディングの基本的な使い方を学ぶ。


### リアクティブデータの定義

コンポーネントの`data`オプションにリアクティブにするデータを定義することができる。
```js
var app = new Vue({
    el: '#app',
    data: {
        message: 'Hello Vue.js!' // messageの変化をVue.jsが監視する
    }
});
```
また、`data`で定義された変数は、オプションのブロック外でも変更を検知できる
```js
const state = {
    count: 0;
};
const app = new Vue({
    el: '#app',
    data: {
        state: state
    }
});
state.count = 1; // Vue.jsが監視しているので、DOMが更新される`
```
`data`で定義されるプロパティは後から追加することはできない。
初期値が定まっていない場合でも、後から使う必要がある場合は空データとして定義しておく必要がある。
この辺りは、オブジェクト指向のインスタンス変数のコンストラクタ内での初期化と似た概念。
```js
var app = new Vue({
    el: '#app',
    data: {
        message: null
    }
});
app.message = 'Hello Vue.js!'; // ここで初めてmessageが定義される
```




## 2.8 テキストと属性のデータバインディング

`マスタッシュ`と呼ばれる`{{}}`で囲まれた式をテキストとして表示することができる。
マスタッシュ記法で書かれた式を`マスタッシュタグ`と呼ぶ。このタグは自動的にバインディングされる。
```html
<div id="app">
    <p>{{ message }}</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        message: 'Hello Vue.js!'
    }
});
```
バインディングでは、ルートに定義されたプロパティだけでなく、ネストされたオブジェクトも参照可能。
```html
<div id="app">
    <p>{{ list[2] }}</p>
    <p> {{ list[index] }}</p>
    <p>{{ list.length }}</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        list: ['りんご', 'ばなな', 'いちご'],
        index: 1
    }
});
```
実際にレンダリングされると、以下のようになる。
```html
<div id="app">
    <p>いちご</p>
    <p>ばなな</p>
    <p>3</p>
</div>
```

マスタッシュはあくまでJSの式をバインディングできるのであって、文をバインディングすることはできない。
```html
<div id="app">
    <p>{{ message = 'Hello Vue.js!' }}</p> <!-- これはエラーになる -->
</div>
```

### 属性のデータバインディング
マスタッシュはテキストデータ固有の記法のため、属性の指定には使えない。
```html
<div id="app">
    <input type="text" value="{{ message }}"> <!-- これはエラーになる -->
</div>
```
属性へのバインドの場合は、`v-bind`ディレクティブを使う。
```html
<div id="app">
    <input type="text" v-bind:value="message">
</div>
```
`v-bind`ディレクティブは、`:`と省略記法がある。
```html
<div id="app">
    <input type="text" :value="message"> <!-- 省略記法:v-bind:value -->
</div>
```
* `v-bind`の修飾子
    * `v-bind`では以下の修飾子をつけることができる
    
    |修飾子|機能|
    |-|-|
    |`.prop`|属性ではなく、DOMプロパティにバインドする。|
    |`.camel`|キャメルケースに変換する。例:property-name → propertyName|
    |`.sync`|双方向バインディングを行う。|

    * `.prop`の例
    ```html
    <div v-bind:text-content.prop="message"></div>
    <div v-bind:scroll-top.prop="scroll"></div>
    ```
    ```js
    var app = new Vue({
        el: '#app',
        data: {
            message: 'Hello Vue.js!',
            scroll: 0
        }
        mounted: function() {
            this.scroll = 100;
        }
    });
    ```

### クラススタイルとデータバインディング

HTMLのclass属性やstyle属性にデータをバインドするとき、オブジェクトや配列を使うことができる。
その際は、一般のHTML同様、値にしたい文字列をキーに指定する
```html
<div id="app">
    <p v-bind:class="{ `child`: isChild, `is-active`: isActive }">Hello Vue.js!</p>
    <p v-bind:style="{ color: textColor, backgroundColor: bgColor }">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        isChild: true,
        isActive: true,
        textColor: 'red',
        bgColor: 'yellow'
    }
});
```
実際にレンダリングされると、以下のようになる。
```html
<div id="app">
    <p class="child is-active">Hello Vue.js!</p>
    <p style="color: red; background-color: yellow;">Hello Vue.js!</p>
</div>
```

* すでにある値と一緒にしようしたい場合
    * 既存の値と併用した場合、`class`属性と`style`属性では挙動が異なる
    * class属性 : 追加される
    * style属性 : 同じプロパティは上書きされる
    ```html
    <div id="app">
        <p class="pre-class" v-bind:class="{ `is-active`: usActive }">Hello Vue.js!</p>
        <p style="color:black" v-bind:style="{ color: textColor }"></p>
    </div>
    ```
    ```js
    var app = new Vue({
        el: '#app',
        data: {
            isActive: true,
            textColor: 'red'
        }
    });
    ```
    実際にレンダリングされると、以下のようになる。
    ```html
    <div id="app">
        <p class="pre-class is-active">Hello Vue.js!</p>
        <p style="color: red;"></p>
    </div>
    ```

* オブジェクトデータの渡し方
```html
<div id="app">
    <p v-bind:class="classObject">Hello Vue.js!</p>
    <p v-bind:style="styleObject">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        classObject: {
            'child': true,
            'is-active': true
        },
        styleObject: {
            color: 'red',
            backgroundColor: 'yellow'
        }
    }
});
```

### 複数属性のデータバインディング
大量のプロパティを従来の方法でバインディングさせようとすると、テンプレートがすごく長くなってしまう。
```html
<div id="app">
    <p 
    v-bind:id="id" 
    v-bind:class="classObject" 
    v-bind:style="styleObject">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        id: 1,
        classObject: {
            'child': true,
            'is-active': true
        },
        styleObject: {
            color: 'red',
            backgroundColor: 'yellow'
        }
    }
});
```

このような場合、`v-bind`の引数を初裏yくしてオブジェクトを渡すことで、簡潔に記述することができる。
```html
<div id="app">
    <p v-bind="bind-item">Hello Vue.js!</p>
    <!--
        オブジェクトをそのまま渡すことが可能
    -->
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        bindItem: {
            id: 1,
            classObject: {
                'child': true,
                'is-active': true
            },
            styleObject: {
                color: 'red',
                backgroundColor: 'yellow'
            }
        }
    }
});
```
引数を持った`v-bind`と併用することで、特定のプロパティに変更を加えることも可能
```html
<div id="app">
    <img v-bind="item" v-bind:id=" 'thumb-' + item.id ">
</div>
```

### SVGのデータバインディング

Vue.jsはSVGのDOMにも対応しているので、簡単に図形操作ができる。

また、SVG自体をコンポーネント化することも可能。




## 2.9 テンプレートにおける条件分岐

Vue.jsはテンプレートベースでの条件分岐にようる描画操作にも対応している。
`v-if`と`v-show`ディレクティブは、付与したHTMLタグの描画に条件を適用する。

どちらも似たような結果を得るが、DOM自体が消えるのと、`display`プロパティの値をいじっているだけと言う違いがある。
```html
<div id="app">
    <p v-if="isShow">Hello Vue.js!</p>
    <p v-show="isShow">Hello Vue.js!</p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        isShow: true
    }
});
```
* `isShow`が`true`の場合、以下のように描画される。
```html
<div id="app">
    <p>Hello Vue.js!</p>
    <p>Hello Vue.js!</p>
</div>
```
* `isShow`が`false`の場合、以下のように描画される。
```html
<div id="app">
    <p></p>
    <p>Hello Vue.js!</p>